# Agent Basics

This notebooks walks you through creating two agents that work together to schedule coffee.

First we create two agents: 

In [2]:
# uncomment and run if running from /docs/agents

# from os.path import dirname
# file_path = dirname(dirname(os.path.abspath('')))
# sys.path.append(file_path)
# # sys.path.append(str(file_path.parent.parent.parent))

# print(sys.path)

In [3]:
from osnap_client.agents import (
  OSNAPBaseAgent, 
  AgentInfo, 
  OSNAPTask, 
  OSNAPAgentRunResponse,
  OSNAPAgentTaskResult
)

def alice_run():
  pass

def alice_start(): 
  pass

def alice_listen(): 
  pass

def alice_complete():
  pass

def alice_terminate():
  pass

class AliceAgent(OSNAPBaseAgent):
  name="alice"
  description="Alice's personal assistant"
  tools=["calendar", "email", "location"]
  id="1234"

  def run(self, task: OSNAPTask):
    return alice_run(self, task)

  def start(self, objective: str, agent_url=None):
    return alice_start(self, objective, agent_url)

  def listen(self, result: OSNAPAgentTaskResult):
    return alice_listen(result)
  
  def complete(self):
    return alice_complete()
  
  def terminate(self):
    return alice_terminate()
  
alice = AliceAgent()

alice.info()

AgentInfo(name='alice', description="Alice's personal assistant", id=1234, tools=['calendar', 'email', 'location'], url=None)

In [25]:
# define the methods so we can test them
def bob_run():
  pass

def bob_start():
  pass

def bob_listen():
  pass

def bob_complete():
  pass

def bob_terminate():
  pass

class BobAgent(OSNAPBaseAgent):
  name="bob"
  description="Bobs's personal assistant"
  tools=["calendar", "email", "location"]
  id="1234"

  def run(self, task: OSNAPTask) -> OSNAPAgentRunResponse:
    return bob_run(task)

  def start(self, objective: str, agent_url=None):
    return bob_start(objective, agent_url)

  def listen(self, result: OSNAPAgentTaskResult):
    return bob_listen(result)
  
  def complete(self):
    return bob_complete()
  
  def terminate(self):
    return bob_terminate()
  
bob = BobAgent()

bob.info()

AgentInfo(name='bob', description="Bobs's personal assistant", id=1234, tools=['calendar', 'email', 'location'], url=None)

In your application you will define the required methods when you create an object, but for the purposes of this tutorial we will define them one at a time as we go. Like the following example:

```python 
def alice_run(task: OSNAPTask):
  print(f"Alice is running task NOW!! {task}")
  
alice.run("something")
```   

First let's say that Alice wants her agent to work with Bob's agent to schedule coffee with him. 
We initialize this request by calling the `start` method on Alice's agent. In our case, we are going to hardcode some logic into Alice's agent to make it easier to understand. 


In [8]:
import openai
import getpass
openai.api_key = getpass.getpass()

bob_info = bob.info()

In [9]:
def alice_start(self, objective, agent_url=None):
    prompt = f"""
    You are {self.description}. You have access to the following tools: {self.tools}.

    Alice wants to {objective}. Bob's agent is {bob_info.description}. Bob's agent has access to the following tools: {bob_info.tools}.

    What are the tasks we need to complete {objective}?
    """

    response = openai.Completion.create(
      engine="text-davinci-003",
      prompt=prompt,
      temperature=0.5,
      max_tokens=200
    )

    return response

In [10]:
response = alice.start(objective="Schedule coffee with Bob")

In [11]:
task_list = response.choices[0].text.split("\n")

In [12]:
task_list

['',
 '1. Identify a date and time for the meeting. ',
 "2. Create an event in Alice's calendar with the agreed upon date and time. ",
 "3. Send an email to Bob's agent to confirm the meeting. ",
 '4. Determine a location for the meeting. ',
 '5. Update the calendar event with the location. ',
 "6. Send an email to Bob's agent to confirm the location."]

The protocol isn't opinionated about WHAT your agent does. In this case we are just picking a hardcoded external agent, and then using it's description and tools directly in the prompt. This allows every agent to define as much logic as they want. The purpose of the protocol is to make them interoperable. 

Now that we have a nice list of tasks, we can start asking Bob's agent for help

In [16]:
prompt = f"""
You are {alice.description}. You have access to the following tools: {alice.tools}.
Alice wants to Schedule coffee with Bob. Bob's agent is {bob_info.description}. Bob's agent has access to the following tools: {bob_info.tools}.

The first task is to {task_list[1]}.

We're going to make a simple request to Bob's agent to help. What is the request?
Format your request as a simple task for Bob's agent to complete. 
You can also specify a tool to use, if you want to be specific.
"""
response = openai.Completion.create(
  engine="text-davinci-003",
  prompt=prompt,
  temperature=0.5,
  max_tokens=200
)

In [17]:
task_map = {}
task_map[response.id] = response.choices[0].text

Now that we have a task ready for Bob's agent, we're going to send it to him. 
We give it an id here so that both agents can keep track of it.

Let's turn this into an OSNAPTaskRequest:

In [35]:
task1 = OSNAPTask(
  objective="Schedule coffee with Bob",
  task_id=response.id,
  task_name=task_map[response.id],
  task_description=task_map[response.id],
  task_tool="calendar"
)

Now we can implement the run method on Bob's agent that we just stubbed out later. 

run takes an OSNAPTask as an input and returns an OSNAPAgentRunResponse. 

Run might be a long running task, so we want to return a status that indicates to the caller if that it's working on it.
Let's just have it return a status of "starting" for now.

In [38]:
def bob_run(task): 
    print("Incoming task from Alice: ", task.task_name)
    return OSNAPAgentRunResponse(
        status="starting",
        message="Sounds good, I'll get started on that.",
        payload={
            "tools": ["calendar"],
        }
    )

bob_run_response = bob.run(task1)

Incoming task from Alice:  
The request is: Please use the calendar to suggest a date and time for Alice and Bob to meet for coffee.
